In [12]:
#import pickle
#import sklearn
#from sklearn import metrics
#from sklearn import linear_model
#import random
#import numpy as np
#from wac import WAC
from tqdm.notebook import tqdm
import pandas as pd
#from sklearn import neural_network
#import matplotlib.pyplot as plt
from nltk.corpus import wordnet as wn
from collections import defaultdict as dd
import nltk 
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /home/crow/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

## Initiate Datasets

In [24]:
wac2vec = pickle.load(open('ddata/clip.bertvocab.embeddings.513.pkl', 'rb'))
len(wac2vec)

30522

In [22]:
concr_scores = pickle.load(open('ddata/AC_ratings_google3m_koeper_SiW.pkl', 'rb'))
len(concr_scores)

2168990

In [29]:
#wn_words = set(i for i in wn.words())
#wn_wac_words = wn_words & set(wac_words)

## ConcreteAbstract Class

In [28]:
class ConcreteAbstract:
    def __init__(self, word_vectors, concr_scores, word_net):
        pass